# Population and Demographic Analysis of Warrants in Connecticut

## Background

## Methods
All of the data is compiled 

##
There are 3 forms of active warrant information provided by the Criminal Justice Policy and Planning Division in Connecticut. 
1. Failure to appear (FTA) warrants for criminal and motor vehicle related claims
2. Violation of probation (VOP) warrants
3. Order to incarcerate (OTI) warrants

In addition, the data gives us the following information:
 - the number of active warrants with law enforcement actions noted
 - the number of warrants issued in just this month
 - the number of served warrants for the month
 - the number of warrants signed by the municipality but served by another jurisdiction
 - total number of active warrants signed by the municipality



### Warrant and Population Demographic Relations

In [1]:
import pandas as pd

In [12]:
# import warrant, demographic, and economic data

warrant_data = pd.read_csv('./Data/WarrantData2019.csv', index_col = 0) # data cleaning done in MonthMerge.py
demo_data = pd.read_csv('./Data/2010_2020_population_changes.csv')
econ_data = pd.read_csv('./Data/EconData.csv', index_col = 0) # data handling done in EconDataConn.py

### Demographic data handling

# remove non-town data
demo_data = demo_data[demo_data['geo_type']=='town'].reset_index(drop = True)

# create new column that combines race and age for pivoting later and rename town name to municipality
demo_data['Demographic'] = demo_data['race_ethnicity']+ ' ' + demo_data['age']
demo_data.rename(columns = {'name':'Municipality'}, inplace = True)

# drop geo_type, 2010, race, age, and pop. change columns
demo_data.drop(['geo_type',' 2010_population ','change_2010_2020','percent_change_2010_2020','race_ethnicity','age'], 
axis = 1, inplace=True)

# pivot the long demo data to wide
demo_data = demo_data.pivot(index = 'Municipality',columns = 'Demographic',values = ' 2020_population ')

# renaming demo data column names so they are much shorter
demo_data.rename(lambda x: x.replace('Hispanic','Hisp.'), axis = 1, inplace = True)
demo_data.rename(lambda x: x.replace('of any race','any race'), axis = 1, inplace = True)
demo_data.rename(lambda x: x.replace('Two or More Races','Multiple Races'), axis = 1, inplace = True)

# merging the dataframes
final_data = pd.merge(warrant_data,demo_data, on = 'Municipality').merge(econ_data, on='Municipality')

Creating annual data from the monthly format given

In [30]:
# get list of months and municipalities to iterate through
from numpy import integer


months = warrant_data['Month'].unique()
municipalities = warrant_data['Municipality'].unique()

# creating annual data from the monthly data we already have in warrant_data
warrant_annual_data = pd.DataFrame(columns = warrant_data.columns.drop('Month'))

for municipalitiy in municipalities:
     annual_data = [municipalitiy]
     for column in warrant_annual_data.columns.drop('Municipality'):
          columnsum = 0
          for month in months:
               columnsum += int(warrant_data.loc[(warrant_data['Month']==month) & (warrant_data['Municipality']==municipalitiy),column])
          annual_data.append(columnsum)
          
     warrant_annual_data.loc[len(warrant_annual_data)] = annual_data
final_annual_data = pd.merge(warrant_annual_data,demo_data, on = 'Municipality').merge(econ_data, on='Municipality')

ValueError: invalid literal for int() with base 10: '4,298'

## Per Capita Visualizations

In [ ]:
import matplotlib.pyplot as plt
import numpy as np